# Mattilsynet

> Vannkvalitet

In [14]:
import deetly
import requests
import json
import pandas as pd
import geopandas as gpd
import pydeck
import os

In [15]:
# Create datastory
package = {
    "name": "Mattilsynet vannforsyning",
    "keywords":["mattilsynet","pydeck"],
}

ds = deetly.story(package)

In [16]:
df_system_data = pd.read_csv('https://data.mattilsynet.no/vannverk/vannforsyningssystem.csv', 
                             sep=';', 
                             converters={'virksomhet_kommunenr':str})
df_system_data.head(1)

,aktiv,orgform,fnr,fnavn,orgnr,virksomhet,virksomhet_kommunenr,virksomhet_kommune,vannforsyningssystem,mtid_vf,...,ant_personer_max,ant_husstander,ant_hytter,forbr_fast_bosetting,forbr_fritidsboliger,forbr_industri,forbr_tjytende,forbr_primnaering,forbr_annet,forbr_lekkasje
0,ja,KOMM,935478715.0,Bærum Kommune,974553619.0,Bærum Kommune Vann Og Avløp,3024,Bærum,Sollihøgda Vannverk Bærum Kommune Vann Og Avløp,Z0704301334220581190KVPHB,...,137.0,53.0,0.0,58.0,0.0,0.0,0.0,0.0,10.0,32.0


In [17]:
df_system = pd.read_csv('https://data.mattilsynet.no/vannverk/vannforsyningssystem_innrapportering.csv', sep=';')
df_system = pd.merge(df_system, 
                     df_system_data[['mtid_vf','virksomhet_kommunenr','virksomhet_kommune','vannforsyningssystem']], 
                     on=['mtid_vf'])
df_system.head(1)

,mtid_vf,periode,vannprod,vannegetnett,vann_mottatt,max_vann_pers,max_vann_dogn,beredsk_oppd,beredsk_ovelse,boliger,...,forbr_industri,forbr_tjytende,forbr_primnaering,forbr_annet,forbr_lekkasje,vannuttak,noedvann_inngaar_alt_kilde,virksomhet_kommunenr,virksomhet_kommune,vannforsyningssystem
0,Z0704301334220581190KVPHB,2009,25355.0,25355.0,0.0,NaN,NaN,ja,ja,ja,...,0.0,0.0,0.0,5.0,32.0,26235.0,NaN,3024,Bærum,Sollihøgda Vannverk Bærum Kommune Vann Og Avløp


In [18]:
df_kommune_prod = df_system[['vannprod','virksomhet_kommunenr']].groupby('virksomhet_kommunenr').sum().reset_index()
df_kommune_prod.columns = ['kommunenummer', 'vannprod']
df_kommune_prod.head(1)

,kommunenummer,vannprod
0,,552869.0


In [19]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json'
df_geo = gpd.read_file(MAP_URL)

In [20]:
df_kommune_vannprod_geo = df_geo.merge(df_kommune_prod, on='kommunenummer')
df_kommune_vannprod_geo.head(1)

,id,kommunenummer,oppdateringsdato,navn,area_kommune,fylke_nummer_2018,fylke_nummer_2019,fylke_navn_2019,fylke_nummer_2020,fylke_navn_2020,geometry,vannprod
0,2,0105,2018-12-10T00:00:00,Sarpsborg,0.063157,01,01,ØSTFOLD,30,VIKEN,"POLYGON ((11.13772 59.41486, 11.18831 59.40549...",1000.0


In [21]:
# Map centered on Trondheim, Norway
view_state_norway = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 45,
    bearing= 0)

In [22]:
layer = pydeck.Layer(
    'GeoJsonLayer',
    data=df_kommune_vannprod_geo,
    stroked = False,
    filled = True,
    extruded = True,
    wireframe = True,
    elevationScale = 0.001,
    get_elevation = 'vannprod',
    width_min_pixels=5,
    get_fill_color='[255, 255, 0.01*vannprod]',
    getLineColor = [
        255,
        255,
        255
    ],
    pickable=True,
    coverage=1
)


tooltip={
        'html':'<b>Kommune: </b>{navn}</br>\
                Fylke: {fylke_navn_2020}</br>\
                Vannproduksjon: {vannprod}',
        'style': {
            'color': 'black',
            'backgroundColor':'#efefef',
            'zIndex': 999
        }
    }

# Render
fig = pydeck.Deck(layers=[layer],
                  map_style="light_no_labels",
                  initial_view_state=view_state_norway,
                  tooltip=tooltip
)

fig.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…

In [24]:
ds.pydeck(fig, "vannkvalitet", showMap=True, tooltip=json.dumps(tooltip))

In [ ]:
df_inntak = pd.read_csv('https://data.mattilsynet.no/vannverk/inntakspunkt_innrapportering.csv', sep=';')
df_inntak.head(1)

In [ ]:
df_behandlingsanlegg = pd.read_csv('https://data.mattilsynet.no/vannverk/vannbehandlingsanlegg_innrapportering.csv', sep=';')
df_behandlingsanlegg.head(1)

In [ ]:
df_levert_andre = pd.read_csv('https://data.mattilsynet.no/vannverk/vann_levert.csv', sep=';')
df_levert_andre.head(1)

In [ ]:
df_distribusjon = pd.read_csv('https://data.mattilsynet.no/vannverk/vanntransportsystem_innrapportering.csv', sep=';')
df_distribusjon.head(1)

In [ ]:
df_inntak_analyse = pd.read_csv('https://data.mattilsynet.no/vannverk/inntakspunkt_analyse.csv', sep=';')
df_inntak_analyse.head(1)

In [ ]:
df_drikkevann_analyse = pd.read_csv('https://data.mattilsynet.no/vannverk/vannforsyningssystem_analyse.csv', sep=';')
df_drikkevann_analyse.head(1)

In [25]:
ds.publish()

Publishing story Mattilsynet vannforsyning to the examples space...
Published at: https://public.deetly.com/examples/5a2ccd0fa0383b291d35a0750cc66b78 

Metadata: https://storage.googleapis.com/deetly/examples/5a2ccd0fa0383b291d35a0750cc66b78/datapackage.json 

